![](problem_statements/E050.png)

# Input

In [1]:
T = int(input().strip())

2


In [2]:
N_list = []
for _ in range(T):
    N_list.append(int(input().strip()))

100
1000


# Steps

The problem will first require a quick primality testing algorithm, so first generate all the primes <= $10^7$, then write a code to test for the numbers >$10^7$.

In [3]:
def get_primes(num):
    "Return a list of all primes <= N"
    
    #Start with the initialization of all numbers from 1 to num as prime=True
    prime = [True if i>=2 else False for i in range(num + 1)]
    
    #Now, run a index from 2 to sqrt(num)
    i = 2
    while i*i<=num:
        #If i is prime, all multiples of i which are <=num will be composite...
        if prime[i]:
            for j in range(i*2,num+1,i):
                prime[j]=False
        i+=1
    #Now, list all the primes - i.e. all n such that prime[n]==True
    ans = []
    for i in range(num+1):
        if prime[i]:
            ans.append(i)
    return ans

In [4]:
primes_list = get_primes(10**7)

We now have a complete list of primes <=$10^7$. It is unlikely we will need primes higher than this as the sum of this list $\approx 3 \times 10^{12}$, which is larger than the maximum $N$. But we will need to test for the primality of the sums, so write a function *check_prime* to test if a number is prime.

In [5]:
def check_prime(num):
    if num<=10**7:
        return num in primes_list
    elif num<=10**14:
        for p in primes_list:
            if num%p==0:
                flag=False
                break
        else:
            flag=True
        return flag
    else:
        return None

Our goal is to find the sequence of primes of maximum length which satisfy two conditions:

1. They must sum to <=$N$
2. That sum must be prime

First  we propose partial solutions by considering only criterion 1. Since we want the sequence to be of maximum length, we'd like the elements in the sequence to be as small as possible. So first, create a sequence starting with $2$, and summing to <=$N$. This is the first partial solution. Suppose the maximum prime in this sequence is $p_i$, we can then start considering sequences including $p_{i+1}$ and higher, shaving off elements at the lower end to keeping the sum <=$N$. These become the second partial solution,and so on. Note that the second partial solution will always be shorter in length than the first - so why consider it? Because in step 2, we will apply primality tests, on these and their sub-sets, so that can change the final length of the complete solutions.

Let's write a generator to yield these proposed partial solutions.

In [6]:
def get_partial_solutions(num):
    start = 0
    end = 1
    S = primes_list[start]
    #First find the end, which has sum(prime_list(start:end))
    while S + primes_list[end]<=num:
        S += primes_list[end]
        end += 1
        
    yield primes_list[start:end],S
    
    #Now, we keep incrementing end, and correspondingly start...
    while end+1<len(primes_list):
        S += primes_list[end]
        end += 1
        while S>num:
            S -= primes_list[start]
            start+=1
        yield primes_list[start:end],S

Now, for a given partial solution, write a definition to search it's contiguous subsets to check condition 2. Take the partial solution:

$2+3+5+7+ \ldots +p_{n-1}+p_n$

Check it's primality - if it is prime, we have a complete solution. If not take two subsets

$[2+3+5+7+ \ldots +p_{n-1}]$
and
$[3+5+7+ \ldots + p_n]$

and check the primality of each, in the specified order. Once we get a prime, return it's value and sequence length. If neither adds to a prime, take smaller subsets, and so on.

In [7]:
def get_complete_solution(partial_solution,S):
    "To search the contgiguous subsets of partial_solution that will sum to a prime"
    L = len(partial_solution)
    #We start dropping terms from either side, until the sum is prime...
    for terms_to_drop in range(L):
        #Scan from left to right...
        for start,end in zip(range(0,terms_to_drop+1),range(L-terms_to_drop,L+1)):
            S_trunc = S - sum(partial_solution[:start]) - sum(partial_solution[end:])
            L_trunc = L - terms_to_drop
            if check_prime(S_trunc):
                return S_trunc,L_trunc

# Solution

Run a generator on *get_partial_solutions*, and for each partial solution, run *get_complete_solution*. Once we have a comlplete solution that is longer than the next partial solution, we can stop the generator. Now among the list of complete solutions, select the longest, and that is our answer.

In [8]:
for N in N_list:
    solution_set = []
    partial_solutions_generator = get_partial_solutions(N)
    #Take the first partial solution..., and get the corresponding complete solution
    solution_set.append(get_complete_solution(*next(partial_solutions_generator)))
    
    #Next partial solution
    ps,S = next(partial_solutions_generator)
    while solution_set[-1][1]<len(ps):
        solution_set.append(get_complete_solution(ps,S))
        ps,S = next(partial_solutions_generator)
        
    ans = max(solution_set,key = lambda x:x[1])
    print(*ans)

41 6
953 21
